## Импорт и установка библиотек

In [1]:
# !pip install dbfread # file reader
# !pip install patool # rar unarchiver
# !pip install linearmodels # econometrics

In [2]:
import pandas as pd
import patoolib
import os
import numpy as np
import shutil

from dbfread import DBF

In [3]:
def read_dbf(file):
    return pd.DataFrame(iter(DBF(file, encoding='cp866')))

## Разархивирование файлов

## Чтение файлов

### Формат файлов отчета 0409802
- Файл PK802yymm.DBF, где yymm – отчетный год, отчетный месяц: файлы «Консолидированный балансовый отчет»
- Файл PS802yymm.DBF, где yymm – отчетный год, отчетный месяц: файлы «Консолидированный балансовый отчет» раздела «Справочно»

In [4]:
# костыль, чтоб нормально внеслись данные за разные года

df_balance = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/0409802')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409802/{filename}', outdir=f'data/0409802/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409802/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409802/{name}/{filename}')
        
        date = name.split('-')[1]
        
        frame.drop(columns=['KORR_P', 'KORR_M', 'VSEGO'], inplace=True) 

        df_balance = pd.concat([df_balance, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/0409802/{name}')

shutil.rmtree(f'data/0409802')
    
df_balance = df_balance.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

INFO patool: ... creating output directory `data/0409802/802-20171001'.
INFO patool: ... creating output directory `data/0409802/802-20180101'.
INFO patool: ... creating output directory `data/0409802/802-20180401'.
INFO patool: ... creating output directory `data/0409802/802-20180701'.
INFO patool: ... creating output directory `data/0409802/802-20181001'.
INFO patool: ... creating output directory `data/0409802/802-20190101'.
INFO patool: ... creating output directory `data/0409802/802-20190401'.
INFO patool: ... creating output directory `data/0409802/802-20190701'.
INFO patool: ... creating output directory `data/0409802/802-20191001'.
INFO patool: ... creating output directory `data/0409802/802-20200101'.
INFO patool: ... creating output directory `data/0409802/802-20200401'.
INFO patool: ... creating output directory `data/0409802/802-20200701'.
INFO patool: ... creating output directory `data/0409802/802-20201001'.
INFO patool: ... creating output directory `data/0409802/802-202

In [5]:
for filename in sorted(os.listdir('data/rars/0409802')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409802/{filename}', outdir=f'data/0409802/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409802/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409802/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            # если оставлять корректировки, то закомментировать нижнюю строку и раскомментировать верхние
            #'KORR_P': 'KORR_P_'+date,
            #'KORR_M': 'KORR_M_'+date,
            #'VSEGO': 'VSEGO'+date
            'VSEGO': date
        })

        # оставляем только колонку "ВСЕГО", если оставляем корректировки, то закомментировать строку
        frame.drop(columns=['KORR_P', 'KORR_M'], inplace=True) 

        df_balance = df_balance.merge(frame, how='left')
        shutil.rmtree(f'data/0409802/{name}')

shutil.rmtree(f'data/0409802')
    
df_balance = df_balance.reset_index(drop=True)

INFO patool: ... creating output directory `data/0409802/802-20171001'.
INFO patool: ... creating output directory `data/0409802/802-20180101'.
INFO patool: ... creating output directory `data/0409802/802-20180401'.
INFO patool: ... creating output directory `data/0409802/802-20180701'.
INFO patool: ... creating output directory `data/0409802/802-20181001'.
INFO patool: ... creating output directory `data/0409802/802-20190101'.
INFO patool: ... creating output directory `data/0409802/802-20190401'.
INFO patool: ... creating output directory `data/0409802/802-20190701'.
INFO patool: ... creating output directory `data/0409802/802-20191001'.
INFO patool: ... creating output directory `data/0409802/802-20200101'.
INFO patool: ... creating output directory `data/0409802/802-20200401'.
INFO patool: ... creating output directory `data/0409802/802-20200701'.
INFO patool: ... creating output directory `data/0409802/802-20201001'.
INFO patool: ... creating output directory `data/0409802/802-202

In [6]:
df_balance.to_csv('data/data_balances.csv')

### Формат файлов отчета 0409803
- Файл PK803yymm.DBF, где yymm – отчетный год, отчетный месяц: файл «Консолидированный отчет о финансовых результатах»

In [7]:
# костыль, чтоб нормально внеслись данные за разные года

df_pnl = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/0409803')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409803/{filename}', outdir=f'data/0409803/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409803/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409803/{name}/{filename}')
        
        date = name.split('-')[1]

        frame.drop(columns=['KORR_P', 'KORR_M', 'VSEGO'], inplace=True) 

        df_pnl = pd.concat([df_pnl, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/0409803/{name}')

shutil.rmtree(f'data/0409803')
    
df_pnl = df_pnl.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

INFO patool: ... creating output directory `data/0409803/803-20171001'.
INFO patool: ... creating output directory `data/0409803/803-20180101'.
INFO patool: ... creating output directory `data/0409803/803-20180401'.
INFO patool: ... creating output directory `data/0409803/803-20180701'.
INFO patool: ... creating output directory `data/0409803/803-20181001'.
INFO patool: ... creating output directory `data/0409803/803-20190101'.
INFO patool: ... creating output directory `data/0409803/803-20190401'.
INFO patool: ... creating output directory `data/0409803/803-20190701'.
INFO patool: ... creating output directory `data/0409803/803-20191001'.
INFO patool: ... creating output directory `data/0409803/803-20200101'.
INFO patool: ... creating output directory `data/0409803/803-20200401'.
INFO patool: ... creating output directory `data/0409803/803-20200701'.
INFO patool: ... creating output directory `data/0409803/803-20201001'.
INFO patool: ... creating output directory `data/0409803/803-202

In [8]:
for filename in sorted(os.listdir('data/rars/0409803')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409803/{filename}', outdir=f'data/0409803/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409803/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409803/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            # если оставлять корректировки, то закомментировать нижнюю строку и раскомментировать верхние
            #'KORR_P': 'KORR_P_'+date,
            #'KORR_M': 'KORR_M_'+date,
            #'VSEGO': 'VSEGO'+date
            'VSEGO': date
        })

        # оставляем только колонку "ВСЕГО", если оставляем корректировки, то закомментировать строку
        frame.drop(columns=['KORR_P', 'KORR_M'], inplace=True) 

        df_pnl = df_pnl.merge(frame, how='left')
        shutil.rmtree(f'data/0409803/{name}')

shutil.rmtree(f'data/0409803')
    
df_pnl = df_pnl.reset_index(drop=True)

INFO patool: ... creating output directory `data/0409803/803-20171001'.
INFO patool: ... creating output directory `data/0409803/803-20180101'.
INFO patool: ... creating output directory `data/0409803/803-20180401'.
INFO patool: ... creating output directory `data/0409803/803-20180701'.
INFO patool: ... creating output directory `data/0409803/803-20181001'.
INFO patool: ... creating output directory `data/0409803/803-20190101'.
INFO patool: ... creating output directory `data/0409803/803-20190401'.
INFO patool: ... creating output directory `data/0409803/803-20190701'.
INFO patool: ... creating output directory `data/0409803/803-20191001'.
INFO patool: ... creating output directory `data/0409803/803-20200101'.
INFO patool: ... creating output directory `data/0409803/803-20200401'.
INFO patool: ... creating output directory `data/0409803/803-20200701'.
INFO patool: ... creating output directory `data/0409803/803-20201001'.
INFO patool: ... creating output directory `data/0409803/803-202

In [9]:
df_pnl.to_csv('data/data_pnls.csv')

### Формат файлов отчета 0409135
- Файл mmyyyy_135_3.DBF, где yymm – отчетный год, отчетный месяц: файл с обязательными нормативами
- https://www.cbr.ru/vfs/credit/formats/135-20210501.pdf

In [10]:
# костыль, чтоб нормально внеслись данные за разные года

df_norms = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/135')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/135/{filename}', outdir=f'data/135/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/135/{name}')
        filename = [fname for fname in folder_contents if fname.upper().endswith('_135_3.DBF')][0]
        frame = read_dbf(f'data/135/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            'REGN': 'REGN_GKO', 
            'C1_3': 'NAME_NORM'
        })
        
        frame.drop(columns=['C2_3', 'C3_3', 'C4_3'], inplace=True) 

        df_norms = pd.concat([df_norms, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/135/{name}')

shutil.rmtree(f'data/135')
    
df_norms = df_norms.reset_index(drop=True).sort_values(['REGN_GKO', 'NAME_NORM'])

INFO patool: ... creating output directory `data/135/135-20171001'.
INFO patool: ... creating output directory `data/135/135-20180101'.
INFO patool: ... creating output directory `data/135/135-20180401'.
INFO patool: ... creating output directory `data/135/135-20180701'.
INFO patool: ... creating output directory `data/135/135-20181001'.
INFO patool: ... creating output directory `data/135/135-20190101'.
INFO patool: ... creating output directory `data/135/135-20190401'.
INFO patool: ... creating output directory `data/135/135-20190701'.
INFO patool: ... creating output directory `data/135/135-20191001'.
INFO patool: ... creating output directory `data/135/135-20200101'.
INFO patool: ... creating output directory `data/135/135-20200401'.
INFO patool: ... creating output directory `data/135/135-20200701'.
INFO patool: ... creating output directory `data/135/135-20201001'.
INFO patool: ... creating output directory `data/135/135-20210101'.
INFO patool: ... creating output directory `data

In [11]:
for filename in sorted(os.listdir('data/rars/135')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/135/{filename}', outdir=f'data/135/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/135/{name}')
        filename = [fname for fname in folder_contents if fname.upper().endswith('_135_3.DBF')][0]
        frame = read_dbf(f'data/135/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            'REGN': 'REGN_GKO', 
            'C1_3': 'NAME_NORM',
            'C2_3': date
        })

        # оставляем только колонку со значениями
        frame.drop(columns=['C3_3', 'C4_3'], inplace=True) 

        df_norms = df_norms.merge(frame, how='left')
        shutil.rmtree(f'data/135/{name}')

shutil.rmtree(f'data/135')
    
df_norms = df_norms.reset_index(drop=True)

INFO patool: ... creating output directory `data/135/135-20171001'.
INFO patool: ... creating output directory `data/135/135-20180101'.
INFO patool: ... creating output directory `data/135/135-20180401'.
INFO patool: ... creating output directory `data/135/135-20180701'.
INFO patool: ... creating output directory `data/135/135-20181001'.
INFO patool: ... creating output directory `data/135/135-20190101'.
INFO patool: ... creating output directory `data/135/135-20190401'.
INFO patool: ... creating output directory `data/135/135-20190701'.
INFO patool: ... creating output directory `data/135/135-20191001'.
INFO patool: ... creating output directory `data/135/135-20200101'.
INFO patool: ... creating output directory `data/135/135-20200401'.
INFO patool: ... creating output directory `data/135/135-20200701'.
INFO patool: ... creating output directory `data/135/135-20201001'.
INFO patool: ... creating output directory `data/135/135-20210101'.
INFO patool: ... creating output directory `data

In [12]:
df_norms.head()

,REGN_GKO,NAME_NORM,20171001,20180101,20180401,20180701,20181001,20190101,20190401,20190701,...,20200701,20201001,20210101,20210401,20210701,20211001,20220101,20230701,20231001,20240101
0,1,Н1.0,18.192,18.157,18.617,18.057,17.733,16.149,16.465,16.797,...,16.971,16.739,18.894,18.100,17.659,19.070,19.446,40.836,39.249,46.328
1,1,Н1.1,14.537,14.114,15.171,14.246,14.386,12.621,13.597,13.837,...,15.677,15.754,16.892,17.181,16.317,17.978,18.139,37.979,36.044,39.475
2,1,Н1.2,14.537,14.114,15.171,14.246,14.386,12.621,13.597,13.837,...,15.677,15.754,16.892,17.181,16.317,17.978,18.139,37.979,36.044,39.475
3,1,Н1.4,NaN,NaN,13.218,12.611,13.099,11.542,11.678,12.224,...,11.840,12.286,13.171,14.178,13.346,14.488,14.464,NaN,NaN,NaN
4,1,Н10.1,0.151,0.150,0.143,0.141,0.141,0.144,0.155,0.150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_norms.to_csv('data/data_norms.csv')

### Формат файлов "Консолидированная финансовая отчетность"
!!! по тому какая форма какой файл я не уверена 

- Файл F815yymm.DBF, где yymm – отчетный год, отчетный месяц: ОФП **=> дополнение в `df_balance`**
- Файл F816yymm.DBF, где yymm – отчетный год, отчетный месяц: ОПУ **=> дополнение в `df_pnl`**
- Файл F817yymm.DBF, где yymm – отчетный год, отчетный месяц: ОДДС

In [14]:
# костыль, чтоб нормально внеслись данные за разные года

df_pnl_early = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/Консолидированная отчетность')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        #patoolib.extract_archive(f'data/rars/Консолидированная отчетность/{filename}', outdir=f'data/Консолидированная отчетность/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/rars/Консолидированная отчетность/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('F816')][0]
        frame = read_dbf(f'data/rars/Консолидированная отчетность/{name}/{filename}')
        
        date = name.split('-')[1]

        frame.drop(columns=['CP', 'NUM_PRIM', 'S_LAST'], inplace=True)

        df_pnl_early = pd.concat([df_pnl_early, frame], join='outer').drop_duplicates()
    
#df_pnl_early = df_pnl_early.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

In [15]:
df_pnl_early.head()

,REGN,RAZDEL,N_PP,N_PP_A,KOD,KOD_PRIZ,KOD_NAME,S_OTCH
0,2015,,1,1,81601.1,1,Процентные доходы,237664.0
1,2015,,2,2,81601.2,1,Процентные расходы,-101918.0
2,2015,,3,3,81601.3,1,Резерв под обесценение кредитов,-47321.0
3,2015,,4,4,81601.4,1,Чистые процентные доходы/(отрицательная процен...,88425.0
4,2015,,5,5,81601.5,1,Чистые непроцентные доходы,37426.0


## Обработка показателей

Рассматриваемые показатели:

1) Отношение собственных средств (34) к чистым активам (34-21)
2) Отношение резервов (2.1.1 - обязательные, 20 - возможные потери, 31.1 - оценочные под ожидаемые кредитные убытки) к чистым активам (34-21)
3) Чистые активы (34-21) или их логарифм
4) _Отношение просроченной задолженности по ссудам к кредитам экономики_
5) _Отношение просроченной задолженности по ссудам к обязательным резервам в ЦБ РФ_
6) _Отношение оборотов по корреспондентским счетам за период времени к чистым активам_
7) Отношение балансовой прибыли (32) к чистым активам (34-21)
8) Отношение чистой прибыли (32.1) к чистым активам (34-21)
9) Норматив текущей ликвидности
10) _Отношение ликвидных активов к чистым активам_
11) _Отношение депозитов физических лиц до 30 дней к депозитам физических лиц_
12) _Отношение объема негосударственных ценных бумаг к чистым активам_

Справка по строкам: https://www.consultant.ru/document/cons_doc_LAW_444612/c1c3277496f8d7c26caf7152797c75ab73d1a385/

In [16]:
def choose_str(df, str, str_col='STR', regn_col='REGN_GKO'):
    return df[df[str_col] == str].drop(columns=str_col)#.set_index(regn_col)

def melt_df(df, value, id_vars=['REGN_GKO'], variable='DATE', to_datetime=True):
    df_remastered = pd.melt(df, id_vars=id_vars).rename(columns={'variable': variable, 
                                                                 'value': value})

    if to_datetime:
        df_remastered['DATE'] = pd.to_datetime(df_remastered[variable], format='%Y%m%d')

    return df_remastered.sort_values([id_vars[0], variable])

In [94]:
# чистая прибыль [прибыль (убыток) за отчетный период]
net_income = melt_df(choose_str(df_balance, '32.1'), 'NET_INCOME')#.fillna(method='ffill').fillna(method='bfill')  

# балансовая прибыль [Прибыль (убыток)]
balance_income = melt_df(choose_str(df_balance, '32'), 'BALANCE_INCOME')#.fillna(method='ffill').fillna(method='bfill')   

# собственные средства
capital = melt_df(choose_str(df_balance, '34'), 'CAPITAL')#.fillna(method='ffill').fillna(method='bfill')   

# резервы
reserves = pd.concat([choose_str(df_balance, '20'),
           choose_str(df_balance, '31.1'), 
           choose_str(df_balance, '2.1.1')]).groupby('REGN_GKO').sum().reset_index()

reserves = melt_df(reserves, 'RESERVES').fillna(method='ffill')#.fillna(method='bfill') 

# обязательства
liabilities = melt_df(choose_str(df_balance, '21'), 'LIABILITIES')#.fillna(method='ffill').fillna(method='bfill')   

# Активы-нетто (чистые активы) - стоимость капитала по рыночной цене минус долговые обязательства.
net_assets = capital.copy().merge(liabilities)
net_assets['NET_ASSETS'] = net_assets['CAPITAL'] - liabilities['LIABILITIES']
net_assets.drop(columns=['CAPITAL', 'LIABILITIES'], inplace=True)

# Н3
liquidity_norm = melt_df(choose_str(df_norms, 'Н3', str_col='NAME_NORM'), 'LIQUIDITY_NORM')

# Н6
risk_norm = melt_df(choose_str(df_norms, 'Н6', str_col='NAME_NORM'), 'RISK_NORM')

# H7
max_norm = melt_df(choose_str(df_norms, 'Н7', str_col='NAME_NORM'), 'MAX_NORM')

# H10.1
inside_norm = melt_df(choose_str(df_norms, 'Н10.1', str_col='NAME_NORM'), 'INSIDE_NORM')

In [95]:
data = reserves.merge(balance_income,  how='left', on=['REGN_GKO','DATE']) \
       .merge(capital,  how='left', on=['REGN_GKO','DATE']) \
       .merge(net_income,  how='left', on=['REGN_GKO','DATE']) \
       .merge(liabilities,  how='left', on=['REGN_GKO','DATE']) \
       .merge(net_assets,  how='left', on=['REGN_GKO','DATE']) \
       .merge(liquidity_norm,  how='left', on=['REGN_GKO','DATE']) \
       .merge(inside_norm,  how='left', on=['REGN_GKO','DATE']) \
       .merge(risk_norm,  how='left', on=['REGN_GKO','DATE']) \
       .merge(max_norm,  how='left', on=['REGN_GKO','DATE'])

cols = ['LIABILITIES', 
        'NET_INCOME', 'BALANCE_INCOME', 'CAPITAL', 'RESERVES', 'NET_ASSETS', 'LIQUIDITY_NORM',
       'RISK_NORM', 'INSIDE_NORM']

data.to_csv('data/initial_data.csv')

In [96]:
data['CAP_TO_NA'] = data.CAPITAL / data.NET_ASSETS
data['RES_TO_NA'] = data.RESERVES / data.NET_ASSETS
data['NA'] = data.NET_ASSETS # np.log(net_assets) # логарифм
# X4
# X5
# X6
data['BI_TO_NA'] = data.BALANCE_INCOME / data.NET_ASSETS
data['NI_TO_NA'] = data.NET_INCOME / data.NET_ASSETS
data['L_NORM'] = data.LIQUIDITY_NORM
#data['R_NORM'] = data.RISK_NORM
data['I_NORM'] = data.INSIDE_NORM
#data['M_NORM'] = data.MAX_NORM
# X10
# X11
# X12

data.drop(columns=cols, inplace=True)

In [97]:
szko_df = pd.read_csv('data/szko.csv', sep=';', index_col=0)#.iloc[:, :22]
szko_df = melt_df(szko_df, 'IS_SZKO')

In [98]:
office_df = pd.read_csv('data/head_offices.csv', sep=';', dtype='str')
office_df = office_df.loc[:, ['cregnum', 'strcuraddr']]

office_df['cregnum'] = office_df['cregnum'].str.replace(" ", "").astype(int)

office_df['strcuraddr'] = office_df.strcuraddr.str.contains('Москва').astype(int)
office_df = melt_df(office_df, 'REGISTERED_MSC', id_vars=['cregnum'], 
                    to_datetime=False).drop(columns='DATE').rename(columns={'cregnum': 'REGN_GKO'})

In [99]:
regressors = data.merge(szko_df, how='left', on=['REGN_GKO','DATE']) \
             .merge(office_df, how='left', on=['REGN_GKO'])

regressors

,REGN_GKO,DATE,MAX_NORM,CAP_TO_NA,RES_TO_NA,NA,BI_TO_NA,NI_TO_NA,L_NORM,I_NORM,IS_SZKO,REGISTERED_MSC
0,1,2017-10-01,127.333,-1.707445,-7.816566,-1999871.0,-0.000000,NaN,278.646,0.151,1.0,1.0
1,1,2018-01-01,121.662,NaN,NaN,NaN,NaN,NaN,224.919,0.150,1.0,1.0
2,1,2018-04-01,129.236,1.009863,4.639846,3381320.0,0.000000,NaN,156.418,0.143,1.0,1.0
3,1,2018-07-01,116.728,1.008256,5.134367,3386710.0,0.000000,NaN,187.032,0.141,1.0,1.0
4,1,2018-10-01,122.906,1.202002,4.828916,2840819.0,0.000000,NaN,199.355,0.141,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1815,3532,2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1816,3532,2021-10-01,NaN,0.000120,-0.000008,-187582874.0,0.000603,NaN,NaN,NaN,NaN,0.0
1817,3532,2023-07-01,NaN,-0.000070,-0.000000,-255167386.0,0.000491,NaN,NaN,NaN,NaN,0.0
1818,3532,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [100]:
regressors.to_csv('data/final_data.csv')